In [1]:
# Import required libraries
import pandas as pd
!pip install jupyter_plotly_dash
import dash
import dash_html_components as html
import dash_core_components as dcc
from dash.dependencies import Input, Output
import plotly.express as px

     |████████████████████████████████| 509 kB 65.8 MB/s eta 0:00:01
     |████████████████████████████████| 51 kB 20.4 MB/s eta 0:00:01
     |████████████████████████████████| 8.0 MB 73.4 MB/s eta 0:00:01
     |████████████████████████████████| 42 kB 3.9 MB/s  eta 0:00:01
     |████████████████████████████████| 95 kB 12.4 MB/s eta 0:00:01
     |████████████████████████████████| 1.0 MB 71.3 MB/s eta 0:00:01
     |████████████████████████████████| 77 kB 18.3 MB/s eta 0:00:01
     |████████████████████████████████| 3.5 MB 64.9 MB/s eta 0:00:01
     |████████████████████████████████| 82 kB 2.0 MB/s  eta 0:00:01
     |████████████████████████████████| 1.8 MB 62.8 MB/s eta 0:00:01
     |████████████████████████████████| 357 kB 77.5 MB/s eta 0:00:01


  Created wheel for dash: filename=dash-1.20.0-py3-none-any.whl size=85841 sha256=96e45d01d29dd7f04ae98b0445155592206ecda347fbf76162542f171e2211e2
  Stored in directory: /tmp/wsuser/.cache/pip/wheels/4b/cb/a6/817d490fb0a1abb2d078b9e9ee835bf7f039d1fd8b9c61b25e
  Created wheel for dash-core-components: filename=dash_core_components-1.16.0-py3-none-any.whl size=3540990 sha256=a9fba669a27ec9a83adfce836093da0b0b020c1b4409f87676b60222134ed651
  Stored in directory: /tmp/wsuser/.cache/pip/wheels/a1/21/97/554773c64b38bf753f189054921c9217174a96d500f15b7f93
  Created wheel for dash-html-components: filename=dash_html_components-1.1.3-py3-none-any.whl size=319490 sha256=5656e270548b22ee651052dece7ef6b4582e9fc6f903c3006e07f301874aa58e
  Stored in directory: /tmp/wsuser/.cache/pip/wheels/f7/eb/55/521dda099c8b09eed0178328cbe10306cb3fee851efd70ae4f
  Created wheel for dash-renderer: filename=dash_renderer-1.9.1-py3-none-any.whl size=1014873 sha256=42e39b1f2d3b0a7a89422de29469aaa5ee7b15c5fb34bde57e268

In [2]:
!wget "https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_dash.csv"


--2022-04-15 23:48:25--  https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_dash.csv
Resolving cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud (cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud)... 198.23.119.245
Connecting to cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud (cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud)|198.23.119.245|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2476 (2.4K) [text/csv]
Saving to: ‘spacex_launch_dash.csv’

spacex_launch_dash. 100%[===================>]   2.42K  --.-KB/s    in 0s      

2022-04-15 23:48:25 (76.3 MB/s) - ‘spacex_launch_dash.csv’ saved [2476/2476]



In [3]:

# Read the airline data into pandas dataframe
spacex_df = pd.read_csv("spacex_launch_dash.csv")
max_payload = spacex_df['Payload Mass (kg)'].max()
min_payload = spacex_df['Payload Mass (kg)'].min()

In [ ]:
# Create a dash application
app = dash.Dash(__name__)

# Create an app layout
app.layout = html.Div(children=[html.H1('SpaceX Launch Records Dashboard',
                                        style={'textAlign': 'center', 'color': '#503D36',
                                               'font-size': 40}),
                                # TASK 1: Add a dropdown list to enable Launch Site selection
                                # The default select value is for ALL sites
                                dcc.Dropdown(id='site-dropdown',
                                            options=[
                                                {'label': 'All Sites', 'value': 'ALL'},
                                                {'label': 'CCAFS LC-40', 'value': 'CCAFS LC-40'},
                                                {'label': 'VAFB SLC-4E', 'value': 'VAFB SLC-4E'},
                                                {'label': 'KSC LC-39A', 'value': 'KSC LC-39A'},
                                                {'label': 'CCAFS SLC-40', 'value': 'CCAFS SLC-40'},
                                            ],
                                            value='ALL',
                                            placeholder="Select a Launch Site here",
                                            searchable=True
                                            ),
                                html.Br(),

                                # TASK 2: Add a pie chart to show the total successful launches count for all sites
                                # If a specific launch site was selected, show the Success vs. Failed counts for the site
                                html.Div(dcc.Graph(id='success-pie-chart')),
                                html.Br(),

                                html.P("Payload range (Kg):"),
                                # TASK 3: Add a slider to select payload range
                                #dcc.RangeSlider(id='payload-slider',...)
                                dcc.RangeSlider(id='payload-slider',
                                                min=0, max=10000, step=1000,
                                                marks={0: '0', 2500: '2500', 5000: '5000', 7500: '7500', 10000: '10000'},
                                                value=[min_payload, max_payload]),
                                # TASK 4: Add a scatter chart to show the correlation between payload and launch success
                                html.Div(dcc.Graph(id='success-payload-scatter-chart')),
                                ])

# TASK 2:
# Add a callback function for `site-dropdown` as input, `success-pie-chart` as output
# Function decorator to specify function input and output
@app.callback(Output(component_id='success-pie-chart', component_property='figure'),
              Input(component_id='site-dropdown', component_property='value'))
def get_pie_chart(entered_site):
    filtered_df = spacex_df
    if entered_site == 'ALL':
        fig = px.pie(filtered_df, values='class', 
        names='Launch Site', 
        title='Total Success Launches By Site')
        return fig
    else:
        # return the outcomes piechart for a selected site
        filtered_df = spacex_df[spacex_df['Launch Site'] == entered_site]
        filtered_df = filtered_df.groupby(['Launch Site', 'class']).size().reset_index(name='class count')
        fig = px.pie(filtered_df, values='class count', 
        names='class',
        title=f'Total Success Launched for site {entered_site}')
        return fig
# TASK 4:
# Add a callback function for `site-dropdown` and `payload-slider` as inputs, `success-payload-scatter-chart` as output
@app.callback(Output(component_id='success-payload-scatter-chart', component_property='figure'),
              [Input(component_id='site-dropdown', component_property='value'), 
              Input(component_id='payload-slider', component_property='value')])
def get_scatter_chart(entered_site, payload):
    low, high = payload
    filtered_df = spacex_df[(spacex_df['Payload Mass (kg)'] > low) & (spacex_df['Payload Mass (kg)'] < high)]       
    if entered_site == 'ALL':
        fig = px.scatter(filtered_df, x='Payload Mass (kg)', y='class', 
        color='Booster Version Category',
        title='Correlation between Payload and Success for all Sites')
        return fig
    else:
        fig = px.scatter(filtered_df[filtered_df['Launch Site'] == entered_site],
        x='Payload Mass (kg)', y='class',
        color='Booster Version Category',
        title=f'Correlation between Payload and Success for site {entered_site}')
        return fig

# Run the app
if __name__ == '__main__':
    app.run_server()

Dash is running on http://127.0.0.1:8050/

 * Serving Flask app '__main__' (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)
